# <font color=blue> Introduction to salabim</font>

<table align="left">
  <tr style="vertical-align:top">    
    <td width="800" style="vertical-align: top">
      <p style="font-family: Arial; font-size: 14px;">
salabim is package for discrete event simulation in Python<br>
Discrete event simulation is a form of parallel programming were alot of things happen at the same time<br><br>         
They key elements in salabim are components which are the entities (or objects) flowing through the system<br>
Our model can be represented by a block diagram through which our components flow (as if they are pin balls)<br>   
A salabim user has compelete control over the machine (process) to determine where, when and how they are moving<br>
This methodology is known as the process driven Discrete Event paradigm and originates from Simula (Nygaard 1967)<br></p>
    </td>
    <td><img src="salabim_logo.png" alt="example image" width="300" height="300" align="right"/></td>  
  </tr>
</table>

<img src="Process_centric.png" alt="process centric" width="500" height="500" align="center"/>

There are two key building blocks essential to using salabim

- The simulation environment that manages the simulation.
- Components with process (class) methods to model system behavior.

#### <font color=green><mark>The simulation environment</font></mark>

A simulation environment is created with sim.Environment(). <br>
The resulting object provides a collection of methods for setting and managing the simulation.

In [2]:
import salabim as sim

env = sim.Environment()

env.run(5)
print("time = ", env.now())

time =  5


The above code snippet creates and runs a simulation of (5 time units)<br> 
But since no components were defined there are no events to step through<br>
Therefor the time jumps immediately from 0 to 5 without performing any tasks<br> 

#### <font color=green><mark>Process methods </font></mark>

System behavior is added trough components interacting with each other through their processes methods.<br>
A process communicates with the simulation environment through the yield statement (a key feature of salabim)<br>
In python the yield statement temporarily suspends a function's execution and saves it's current state<br>

A salabim component inherits from sim.Component() and as such inherits all functionality<br>
Normally a salabim component class has its own process method which starts automatically upon env.run()<br>

It requires at least one yield statement in order to hand control to the salabim time event mechanism<br>
In below example there is one yield (hold), which hands control to the event scheduler <br>
The component is then placed on the FEL which turns it into a 'scheduled' state until one (1) time unit has passed.<br>
It then returns to the process of the component and continues the code in a so called 'current' state<br>


In the below process method there are no more statements after the yield statement<br>
Thus the process method terminates and the component gets into a so called state of 'data'. <br>
This means the process method no longer runs (unless the component is reactivated later).<br>

In [3]:
# import statement of salabim with alias sim
import salabim as sim

#Definition of a 'component' inheriting from the Component class 
class Person(sim.Component):
    
    # the process method automatically runs once the simulation starts i.e. env.run() 
    def process(self):        
        yield self.hold(1)
        
# each salabim simulation needs a environment        
env=sim.Environment(trace=True)

Person()

# the simulation is run through this statement.
env.run(10)
        

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               4157391535.py
   12                                  default environment initialize       
   12                                  main create                          
   12       0.000 main                 current                              
   14                                  person.0 create                      
   14                                  person.0 activate                    scheduled for 0.000 @    8+ process=process
   17                                  main run +10.000                     scheduled for 10.000 @   17+
    8+      0.000 person.0             current                              
    9                                  person.0 hold +1.000                 scheduled f

When introducing a infinite while True loop the person keeps on walking and resting as below:

In [5]:
import salabim as sim

class Person(sim.Component):     
    def process(self):        
        while True:
            yield self.hold(2,mode='walk')
            yield self.hold(0.5,mode='rest')

env=sim.Environment(trace=True)

Person()

env.run(5)

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               53082488.py
    9                                  default environment initialize       
    9                                  main create                          
    9       0.000 main                 current                              
   11                                  person.0 create                      
   11                                  person.0 activate                    scheduled for 0.000 @    4+ process=process
   13                                  main run +5.000                      scheduled for 5.000 @   13+
    4+      0.000 person.0             current                              
    6                                  person.0 hold +2.000                 scheduled for 

The process method may be named differently but then needs to be initialised differently as per below:

In [10]:
import salabim as sim

class Person(sim.Component):
    def walking(self):        
        yield self.hold(1)       
   
env=sim.Environment(trace=True)

Person(process='walking')

env.run(10)

line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               3293430043.py
    7                                  default environment initialize       
    7                                  main create                          
    7       0.000 main                 current                              
    9                                  person.0 create                      
    9                                  person.0 activate                    scheduled for 0.000 @    4+ process=walking
   11                                  main run +10.000                     scheduled for 10.000 @   11+
    4+      0.000 person.0             current                              
    5                                  person.0 hold +1.000                 scheduled f

## <font color=blue> Concept of State, process interaction and FEL</font>

Components in Salabim can only be in one of these eight states: <br>

- data
- current
- scheduled
- passive
- requesting
- waiting
- standby
- interrupted

For a component to change between states the following process interaction methods are defined.<br>
These (component) methods have a lot of functionality which cannot be fully covered here, in simple terms:<br>


<table align="left" style="font-family: Arial; font-size: 14px;">
<thead>
<tr><th style="text-align:left;">Method</th><th style="text-align:left;">Description</th></tr>
</thead>
<tbody>
    
<tr><td style="text-align:left;">hold()</td><td width="800" style="vertical-align:top; text-align:left;">
This will set the component into a scheduled state and add it to the FEL
</td></tr> 
    
<tr><td style="text-align:left;">cancel()</td><td style="text-align:left;">
This will remove the component from the FEL (if present) and set into a 'data' state
</td></tr>
    
<tr><td style="text-align:left;">activate()</td><td style="text-align:left;">
This sets a component onto the FEL in a scheduled state for the last event time<br>
When activated it continues from its (interrupted) process interaction yield statement.<br>
If the component was 'data' the process method is reactivated and the component turns live    
</td></tr>
    
<tr><td style="text-align:left;">passivate()</td><td style="text-align:left;">
Passivate is the way to make a, usually current, component passive.<br>
This is essentially the same as calling yield self.hold(duration=inf)
</td></tr>    
    
<tr><td style="text-align:left;">request()</td><td style="text-align:left;">
Request has the effect that the component will check whether the requested quantity from a resource is available.
</td></tr>
    

<tr><td style="text-align:left;">interrupt()</td><td style="text-align:left;">
With interrupt components that are not current or data can be temporarily be interrupted.<br>
</td></tr>
    
<tr><td style="text-align:left;">resume()</td><td style="text-align:left;">
Once a resume is called for the component, the component will continue<br>
For scheduled with the remaining time, for waiting or requesting possibly with the remaining fail_at duration.</td></tr>   
    
    
<tr><td style="text-align:left;">standby()</td><td style="text-align:left;">
Standby has the effect that the component will be triggered on the next simulation event.
</td></tr>               
    
<tr><td style="text-align:left;">wait()</td><td style="text-align:left;">
Wait has the effect that the component will check whether the value of a state meets a given condition
</td></tr>     
    
</tbody>
</table>
<br>

#### <font color=blue>Future Event List (FEL)</font>

The mechanism that drives the scheduling of the events in salabim is the Future Event List <br>
This is a list (technically a heapq) of events sorted by a tuple (time, priority, sequence) where: <br>

- time is the activation time
- priority is the given priority
- sequence is a continously increasing counter for each placement on the future event list). <br>
  When urgent is applied, the sequence is negated.
  
The FEL is not intended for salabim users but can be very usefull for understanding its inner workings<br>
The contents of the FEL can be viewed by calling env._event_list <br>

Only when the time of the next event of a component is known it is placed on the (FEL) <br>
Most commonly this is 'scheduled', but could also be a 'requesting' or 'waiting' state (with a fail_at or fail_delay) <br>

The components on the FEL are ordered according to their (re-activation) time and priority <br>
The system state can be changed in between the activation from one component on the FEL to the next <br>
In this instanteneous (or current) state, new (future) events can be created and variables or attributes set<br>
When the component changes its 'current' state (by inevitably encountering a yield statement within its process)<br>
The FEL is updated with the new events (if generated) and finally the first component on the FEL is re-activated <br>
The component then resumes its process (where it left from the yield) in a 'current' state and repeats the process.


### <font color=green><mark>Defined State: Data</font></mark>

**For a component to be in the state of 'data' just means that its process method is no longer active<br>**
It either reached the end of its instructions or it was cancelled by another component

In below example there are two component clases defined <br>
A Person class which is defined to be in a scheduled state from 0 to 3<br>
At time=3 the instantiated "Person" p1 terminates and becomes 'data '<br>
A Checker class which is in a scheduled state every 1 time unit<br>
The instantiated checker prints uit the state of p1 each time it becomes current (after 1 time unit)<br>


In [16]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        yield self.hold(3)
        
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the state of {p0} is {p0.status()}')

env=sim.Environment(trace=True)

p0=Person()

Checker()

env.run(5)


line#        time current component    action                               information
------ ---------- -------------------- -----------------------------------  ------------------------------------------------
                                       line numbers refers to               4105219571.py
   13                                  default environment initialize       
   13                                  main create                          
   13       0.000 main                 current                              
   15                                  person.0 create                      
   15                                  person.0 activate                    scheduled for 0.000 @    4+ process=process
   17                                  checker.0 create                     
   17                                  checker.0 activate                   scheduled for 0.000 @    8+ process=process
   19                                  main run +5.000                  

The person p0 can be reactivated only through the method(s) other component(s) (it cannot reactivate it self)<br>
Below we added some logic in the checker class to activate p0 at time=5 if it is 'data'<br>
We now let the simulation run until time=10 

In [18]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        yield self.hold(3)
        
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the state of {p0} is {p0.status()}')
            
            if env.now()==5 and p0.status()=='data':
                p1.activate()

env=sim.Environment()

p0=Person()

Checker()

env.run(10)


At time 1.0 the state of Person (person.0) is scheduled
At time 2.0 the state of Person (person.0) is scheduled
At time 3.0 the state of Person (person.0) is data
At time 4.0 the state of Person (person.0) is data
At time 5.0 the state of Person (person.0) is data
At time 6.0 the state of Person (person.0) is scheduled
At time 7.0 the state of Person (person.0) is scheduled
At time 8.0 the state of Person (person.0) is data
At time 9.0 the state of Person (person.0) is data
At time 10.0 the state of Person (person.0) is data


### <font color=green><mark>Defined State: Current</font></mark>

The 'current' state in salabim is an important concept to understand.

- It is 'instantenous' and where all the progam logic is executed 
- It is where components interact and can change (each others) 'state'
- It takes place before the next progression in time is made on the FEL
- It is never needed (nor called) by the user as it is only a consequence of certain process interaction(s)

**A component can get into a current state:**
    
 - After it comes out of a scheduled state (from the FEL)
 - When it is triggered or 'activated' by another component    
   
Note that there can only be one component in a current state at any moment in time!!

<font color=red> *Note: When all instantaneous logic is exhausted for a particular component it needs to change its 'current' state state before the next component on the FEL (with same activation time) can be re-activated (into its own current state) In this manner the current state can be transferred between components before a component on the FEL has a later activation time. This is the driver of the progressesion of (the simulation) time.</font>


Take the following example, only when p0 comes out of its hold will it be in a current state<br>
For an "instantenous" moment, it prints out its state and then returns to the scheduled hold state.<br>

In [1]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        self.number_of_holds=0
        while True:
            yield self.hold(1)     
            print(f'I am {self.status()}')     
            
env=sim.Environment()

p0=Person()

env.run(3)

I am current
I am current
I am current


The current state can only end when another process interaction yield statement is encountered within its process.<br>
In below example after p1 comes out of its firsts hold it is 'current' but immediately thereafter becomes 'scheduled'<br>

In [2]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        self.number_of_holds=0
        while True:
            yield self.hold(1)     
            print(f'At time {env.now()} the state of {self.name()} is {self.status()}')    
            yield self.hold(1) 
            
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the state of {p0.name()} is {p0.status()}')    

env=sim.Environment()

p0=Person()
Checker()

env.run(5)

At time 1.0 the state of person.0 is current
At time 1.0 the state of person.0 is scheduled
At time 2.0 the state of person.0 is scheduled
At time 3.0 the state of person.0 is current
At time 3.0 the state of person.0 is scheduled
At time 4.0 the state of person.0 is scheduled
At time 5.0 the state of person.0 is current
At time 5.0 the state of person.0 is scheduled


### <font color=green><mark>Defined State: Scheduled</font></mark>

**A component is scheduled when it is known in advance when its next event takes place <br>**
In salabim a component can become become scheduled through the methods hold, activate and resume<br>

Each component that hits a yield with any of these methods is added to the FEL <br>
All components on the FEL are then (re)ordered for activation time and priority<br>


As an example we show Persons which are instantiated by the Checker process
Each person is activated per time increment and this is seen in the FEL 

In [7]:
import salabim as sim

class Person(sim.Component):
    def process(self):
        yield self.hold(1)            
            
class Checker(sim.Component):
    def process(self):           
        while True:
            Person()
            print(f'At time {env.now()} FEL {[(i[0],i[1],i[2],i[3].name()) for i in env._event_list]}')   
            yield self.hold(1)   
            
env=sim.Environment()

Checker()

env.run(5)

At time 0.0 FEL [(0.0, 0, 3, 'person.0'), (5, inf, 2, 'main')]
At time 1.0 FEL [(1.0, 0, 5, 'person.0'), (5, inf, 2, 'main'), (1.0, 0, 6, 'person.1')]
At time 2.0 FEL [(2.0, 0, 8, 'person.1'), (5, inf, 2, 'main'), (2.0, 0, 9, 'person.2')]
At time 3.0 FEL [(3.0, 0, 11, 'person.2'), (5, inf, 2, 'main'), (3.0, 0, 12, 'person.3')]
At time 4.0 FEL [(4.0, 0, 14, 'person.3'), (5, inf, 2, 'main'), (4.0, 0, 15, 'person.4')]
At time 5.0 FEL [(5.0, 0, 17, 'person.4'), (5, inf, 2, 'main'), (5.0, 0, 18, 'person.5')]


Note that the component 'main' is automatically generted and controls the simulation, it will be activated at time=5

### <font color=green><mark>Defined State: Passive</font></mark>

**A component is passive when it is scheduled for time=inf which is essentially in a infenite hold.<br>**
A component can passivate itself (as below) or can be passivated by another component.<br>
Reactivation can only be done by another component (it cannot 'wake'itself up from sleep)<br>
When the state of the component is scheduled, it is removed from the FEL


In order to show the passive state of p0 another process i.e. component 'checker' is instantiated<br>
This process also re-activates p0, which stays in a sleeping state at line 7, until it is (re) activated.<br>

In [3]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        while True:
            yield self.hold(1)                 
            yield self.passivate()            
            print(f'At time {env.now()} {self.name()} was woken up, its status is: {self.status()}')                 
            
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(3)   
            print(f'At time {env.now()} {self.name()} activated {p0.name()} from state: {p0.status()}')      
            p0.activate()
            
env=sim.Environment()

p0=Person()
checker=Checker()
env.run(6)

At time 3.0 checker.0 activated person.0 from state: passive
At time 3.0 person.0 was woken up, its status is: current
At time 6.0 checker.0 activated person.0 from state: passive
At time 6.0 person.0 was woken up, its status is: current


### <font color=green><mark>Defined State: Requesting</font></mark>

A component can only be in a 'requesting' state when <br>

**Its request() method is called on an instance of the sim.Resource class**<br>

A resource needs to be instantiated as follows:<br>
- myresource=sim.Resource('myresource')<br>
Then the request can be made by the component:<br>
- yield self.request(myresource)<br>

A resource is something that is in short supply and that needs to be 'requested' by a component<br>
When this is not (sufficently) available, the component remains in a requesting state<br>
But when sufficient capacity becomes available it continues from its yield and becomes 'current'<br>
Note that when a component enters a yield request it has no effect on the FEL<br>
The component is not 'scheduled' as it does not know in advance when it will be activated<br>

Resources are a powerful way of process interaction (cannot be fully covered here) and are defined as per below:

In [4]:
import salabim as sim

class Person(sim.Component):
    def process(self):         
        yield self.hold(1)                 
        yield self.request(ticket_office)    
        yield self.hold(1,mode='buying ticket') 
        self.release()            
            
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the status of {p0.name()} is: {p0.status()}')  
            if env.now()==3:
                ticket_office.set_capacity(cap=1)
                print(f'At time {env.now()} {self.name()} set the ticket_office capacity to {ticket_office.capacity()}')               
          
env=sim.Environment()

p0=Person()
checker=Checker()

ticket_office=sim.Resource(name='ticket_office',capacity=0)

env.run(5)

At time 1.0 the status of person.0 is: requesting
At time 2.0 the status of person.0 is: requesting
At time 3.0 the status of person.0 is: requesting
At time 3.0 checker.0 set the ticket_office capacity to 1
At time 4.0 the status of person.0 is: scheduled
At time 5.0 the status of person.0 is: data


In above example at time=3 the capacity of the resource is increased to 1<br>
This causes p0 to be able to claim from the resource at time=3 to get status 'current'<br>
It immediately goes into a scheduled state when it encounters the yield self.hold(1) to buy the ticket<br>
After it buys the ticket the resource is released (so its no longer claiming the resource)<br>
Because it reaches the end of its process method p1 now has become status 'data<br>

### <font color=green><mark>Defined State: Waiting</font></mark>

A component can only be in a 'waiting' state when 

**Its wait() method is called on an instance of the sim.State class**<br>

A state needs to be instantiated as follows:<br>
- mystate=sim.State('mystate')<br>
Then the component can wait for mystate to reach a certain condition: <br>
- yield self.wait(mystate)<br>

Each time the value of mystate is (re)set within the program the yield wait is checked.<br>
When the value of the state meets the condition the component becomes current<br>
If the value does not meet the condition the component remains in state of 'waiting' <br>
In this context mystate is (default) a boolean but can be anything (string/float/int etc)<br>

States are a powerful way of process interaction (cannot be fully covered here)

In [5]:
import salabim as sim

class Person(sim.Component):
    def process(self):         
        yield self.hold(1)                 
        yield self.wait(ticket_office)    
        yield self.hold(1,mode='buying ticket') 
        ticket_office.reset()
            
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the status of {p0.name()} is: {p0.status()}')  
            if env.now()==3:
                ticket_office.set()
                print(f'At time {env.now()} {self.name()} sets the ticket_office capacity to {ticket_office()}')               
          
env=sim.Environment()

p0=Person()
checker=Checker()

ticket_office=sim.State(name='ticket_office')

env.run(5)

At time 1.0 the status of person.0 is: waiting
At time 2.0 the status of person.0 is: waiting
At time 3.0 the status of person.0 is: waiting
At time 3.0 checker.0 sets the ticket_office capacity to True
At time 4.0 the status of person.0 is: scheduled
At time 5.0 the status of person.0 is: data


### <font color=green><mark>Defined State: Standby</font></mark>

**A component can only be in a 'standby' state when a yield self.standby() method is called.** <br>

This has the effect that the component will continue from its yield for each simulation event.<br>
If another event happens the component will become 'current' otherwise it will remain in a state of 'standby'

This can be a very efficient way to check for a certain condition but ut can be expensive in terms of cpu <br>

In below example mycar has no events scheduled it is only in standby mode waiting for other events to happen<br>
Each time a simulation event of mybike occurs it releases mycar from its 'standby' state into a 'current' state<br>
Note that 'the current' state is instantanous as it immediately loops back into a state of 'standy'

In [22]:
import salabim as sim

class Car(sim.Component):   
    def process(self): 
        while True:
            yield self.standby(mode='waiting')
            print(f'At time {env.now()} {self.name()} came out of standby') 
            
class Bike(sim.Component):
    def process(self): 
        while True:
            yield self.hold(1,mode='drive') 
            yield self.hold(1,mode='stop') 

env=sim.Environment()
mycar=Car()
mybike=Bike()
            
env.run(3)

At time 0.0 car.0 came out of standby
At time 1.0 car.0 came out of standby
At time 2.0 car.0 came out of standby
At time 3.0 car.0 came out of standby


### <font color=green><mark>Defined State: Interrupted</font></mark>

Components can be interrupted with the yield self.interrupt() method.<br>
This results in a component 'freezing' its process until it is resumed with the self.resume() method.<br>
When the resume method is called the component continues where it was frozen in time, this can be: <br>

- For a scheduled state  with the remaining time
- for waiting or requesting possibly with the remaining fail_at duration

As with passivate/activate, this can only be done by another component<br>
Note that you can have 'stacked' interrupts which each need to be resumed before the component can continue:

In [9]:
import salabim as sim

class Person(sim.Component):
    def process(self):        
        yield self.hold(5)               
        
class Checker(sim.Component):
    def process(self):   
        while True:
            yield self.hold(1)   
            print(f'At time {env.now()} the state of {p0} is {p0.status()}')
            
            if env.now()==1:
                p0.interrupt()
                print(f'At time {env.now()} the state of {p0} is {p0.status()}')
            
            if env.now()==3 and p0.status()=='interrupted':
                p0.resume()
                print(f'At time {env.now()} the state of {p0} is {p0.status()}')

env=sim.Environment()

p0=Person()

Checker()

env.run(5)


At time 1.0 the state of Person (person.0) is scheduled
At time 1.0 the state of Person (person.0) is interrupted
At time 2.0 the state of Person (person.0) is interrupted
At time 3.0 the state of Person (person.0) is interrupted
At time 3.0 the state of Person (person.0) is scheduled
At time 4.0 the state of Person (person.0) is scheduled
At time 5.0 the state of Person (person.0) is scheduled
